In [1]:
%pip install ipympl mplcursors

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# ✅ Plotly가 화면에 표시될 수 있는 렌더러를 자동으로 고릅니다.
import plotly.io as pio
import plotly.graph_objects as go
import sys, os

# 환경별 추천 순서
candidates = []

# VS Code 노트북
if os.environ.get("VSCODE_PID"):
    candidates = ["vscode", "notebook_connected", "notebook", "jupyterlab", "browser", "iframe"]
# Colab
elif "google.colab" in sys.modules:
    candidates = ["colab", "notebook_connected", "notebook", "jupyterlab", "browser", "iframe"]
# JupyterLab/Notebook 추정
else:
    candidates = ["notebook_connected", "notebook", "jupyterlab", "vscode", "browser", "iframe"]

picked = None
for r in candidates:
    try:
        pio.renderers.default = r
        picked = r
        break
    except Exception:
        continue

print(f"✅ Plotly renderer = {picked}")

# 🔎 작은 테스트 차트 (이게 보이면 렌더러 OK)
fig = go.Figure(data=[go.Scatter(x=[2018, 2019, 2020], y=[1, 2, 3], mode="lines+markers", name="test")])
fig.update_layout(title=f"Renderer test — {picked}")
fig.show()


✅ Plotly renderer = notebook_connected


In [3]:
import re
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# 1) 데이터 로드
#  - 깃허브 원본 CSV를 직접 읽어 즉시 재현 가능한 파이프라인을 만든다.
#  - 외부 URL로부터 읽을 때는 스키마가 바뀔 가능성이 있어, 이후 단계에서
#    컬럼 존재 여부를 항상 방어적으로 체크한다.
# -------------------------------------------------------------------
URL = "https://raw.githubusercontent.com/HyeokjuCHu/WAR_Predict/refs/heads/master/kbo_dataset_2018_2024.csv"
df = pd.read_csv(URL)

# -------------------------------------------------------------------
# 2) 컬럼 명 상수화
#  - 하드코딩 문자열을 상수로 빼두면, 원본 스키마가 바뀌었을 때
#    코드 전체를 뒤지지 않고 상수만 수정해서 대응 가능하다(유지보수성↑).
# -------------------------------------------------------------------
COL_PLAYER = "player_name"
COL_TEAM   = "team"
COL_YEAR   = "year"

# 카운팅 스탯/비율 지표 (관심 지표를 한곳에 모아 관리)
COL_SO   = "SO"             # 삼진(카운트형)
COL_HR   = "HR"             # 홈런(카운트형)
COL_AB   = "AB"             # 타수(모수)
COL_H    = "H"              # 안타(모수의 일부)
COL_AVG0 = "batting_avg"    # 원본 타율(있으면 그대로 사용)

# -------------------------------------------------------------------
# 3) 팀명 정규화
#  - 원본 데이터에 '롯데', 'LOTTE', 'Lotte' 등 표기가 혼재할 수 있으므로
#    단일 표기('Lotte')로 통일한다. 이렇게 해야 필터가 정확히 작동한다.
# -------------------------------------------------------------------
def norm_team(s):
    s = str(s)
    return "Lotte" if "Lotte" in s or "LOTTE" in s or "롯데" in s else s

# -------------------------------------------------------------------
# 4) 연도/팀 필터링
#  - 연도는 수치형으로 강제 변환(errors='coerce'는 변환 실패를 NaN으로 처리)
#    → 잘못된 값이 섞여도 다운스트림에서 안전하게 거를 수 있다.
#  - 팀은 위에서 정의한 규칙으로 정규화한 뒤, 롯데 + 2018~2024만 남긴다.
# -------------------------------------------------------------------
df[COL_YEAR] = pd.to_numeric(df[COL_YEAR], errors="coerce")
df[COL_TEAM] = df[COL_TEAM].map(norm_team)
base = df[(df[COL_TEAM]=="Lotte") & (df[COL_YEAR].between(2018, 2024))].copy()

# -------------------------------------------------------------------
# 5) 수치형 변환(존재하는 컬럼만)
#  - 외부 데이터엔 누락/문자열 혼입이 잦다. 집계/연산 전에 숫자로 정리해두면
#    집계 시 형변환 오류를 방지할 수 있다. (errors='coerce'로 NaN 방치)
# -------------------------------------------------------------------
for c in [COL_SO, COL_HR, COL_AB, COL_H, COL_AVG0]:
    if c in base.columns:
        base[c] = pd.to_numeric(base[c], errors="coerce")

# -------------------------------------------------------------------
# 6) 표시용 이름(legend_name) 설계
#  - 발표/시각화에서 사람이 알아보기 쉬운 표기를 쓰되,
#    원본 데이터의 실제 표기와 자동 매칭해 휴먼에러를 줄인다.
#  - 단순 문자열 일치로는 철자 변형/이니셜 등에 약하므로 토큰 기반 매칭을 사용.
# -------------------------------------------------------------------
tokens = {
    "Jeon Jun-woo": ["jeon","jun","woo"],
    "Jung Hoon":    ["jung","hoon"],
    "Han Dong-Hee": ["han","dong","hee"],
}

# -------------------------------------------------------------------
# 7) 토큰 기반 안전 매칭
#  - 각 선수에 대해: player_name을 소문자화한 뒤, 토큰이 몇 개 포함되는지 카운트
#  - '2개 이상' 일치 조건을 주어 우연 일치(동명이인/잡음)를 완화
#  - 후보가 여러 개인 경우 .mode().iloc[0]으로 최빈값 선택(데이터 흔들림에 강함)
#  - 결과: 표시이름(disp) → 데이터셋 실제 이름(real) 매핑(name_map)
# -------------------------------------------------------------------
name_map = {}  # 표시이름 -> 실제 데이터셋의 player_name
for disp, toks in tokens.items():
    k = base[COL_PLAYER].astype(str).str.lower().apply(lambda s: sum(t in s for t in toks))
    cand = base.loc[k >= 2, COL_PLAYER]
    if not cand.empty:
        name_map[disp] = cand.mode().iloc[0]

print("Resolved mapping (legend name -> dataset name):")
for k, v in name_map.items():
    print(f"  {k} -> {v}")

# -------------------------------------------------------------------
# 8) 시각화/집계에 쓸 legend_name 컬럼 생성
#  - 원본 player_name을 기본값으로 두고, 매핑에 성공한 경우에만 사람이 읽기 쉬운
#    표시 이름으로 치환한다. (원본 보존 + 가독성 확보)
# -------------------------------------------------------------------
base["legend_name"] = base[COL_PLAYER]
for disp, real in name_map.items():
    base.loc[base[COL_PLAYER]==real, "legend_name"] = disp

# -------------------------------------------------------------------
# 9) 최종 분석 대상 서브셋
#  - legend_name이 우리가 찾은 3인에 속하는 행만 남긴다.
#  - 매핑에 실패한 선수가 있으면 해당 이름은 자동으로 제외(강건성).
# -------------------------------------------------------------------
target = base[base["legend_name"].isin(list(name_map.keys()))].copy()

# -------------------------------------------------------------------
# 10) 타율(AVG) 생성 로직
#  - 원본 타율 컬럼(batting_avg)이 있으면 그것을 신뢰하고 사용(일관성).
#  - 없거나 NaN이면 H/AB로 계산한다.
#    * 분모 0(AB=0)인 경우는 NaN 처리하여 잘못된 무한/0나누기 방지.
#    * .get을 쓰는 이유: 컬럼이 전혀 없을 때도 KeyError 없이 동작(방어적 코딩).
# -------------------------------------------------------------------
if COL_AVG0 in target.columns and target[COL_AVG0].notna().any():
    target["AVG"] = target[COL_AVG0]
else:
    target["AVG"] = np.where(target.get(COL_AB, 0) > 0,
                             target.get(COL_H, 0) / target.get(COL_AB, 1),
                             np.nan)


Resolved mapping (legend name -> dataset name):
  Jeon Jun-woo -> Jun-woo Jeon
  Jung Hoon -> Hoon Jung
  Han Dong-Hee -> Dong Hui Han


In [4]:
import plotly.express as px
import plotly.io as pio

def line_chart_px(ycol: str, title: str, ytitle: str, yfmt: str = "int"):
    """
    목적:
      - 선수별 연도 추이를 선그래프로 비교.
      - 발표 시 툴팁과 축 포맷을 선수 지표 특성에 맞게 가독성 최적화.

    매개변수:
      ycol  : target에 존재하는 y축 지표 컬럼명 (예: "SO", "HR", "AVG")
      title : 그래프 제목
      ytitle: y축 제목(축 라벨)
      yfmt  : 값 포맷. "int"는 정수지표(SO, HR 등), "avg"는 소수 셋째자리(AVG 등)

    설계 포인트:
      - 결측 제거(dropna)로 툴팁/렌더링 오류 예방.
      - 연도 정렬로 ‘시간 흐름’이 시각적으로 자연스럽게 보이도록.
      - category_orders를 지정해 x축 카테고리 순서를 고정(자료 순서에 흔들리지 않음).
      - hovertemplate로 발표용 툴팁을 간결/정확하게(불필요한 정보 제거).
      - hovermode="x unified"로 동일 x위치의 선수 값을 한 번에 비교(발표시 설명이 쉬움).
      - template="plotly_white"로 밝은 배경 → 프로젝터/발표 화면 가독성↑.
    """
    # 1) 결측 제거: y값이 NaN인 행을 제외
    #    - Plotly는 NaN 처리에 관대하지만, 툴팁/라인 끊김 등 시각적 잡음을 줄이기 위해
    #      미리 정리한다.
    d = target.dropna(subset=[ycol]).copy()

    # 2) 연도 기준 정렬
    #    - 시간이 뒤섞여 있으면 선이 지그재그로 연결되어 추세 해석이 어려움.
    d = d.sort_values(COL_YEAR)

    # 3) x축 카테고리 순서 고정
    #    - Plotly는 범주형 x축의 경우 데이터 등장 순서에 의존하기도 한다.
    #      unique() → list → 정렬로, 2018→…→2024 순서 보장.
    category_orders = {COL_YEAR: sorted(d[COL_YEAR].unique().tolist())}

    # 4) 선그래프 생성
    #    - color="legend_name": 선수별로 색을 다르게 분리하여 비교를 직관화.
    #    - markers=True: 각 시즌 포인트를 점으로 강조해 결측/변곡을 쉽게 식별.
    fig = px.line(
        d,
        x=COL_YEAR, y=ycol, color="legend_name",
        markers=True,
        category_orders=category_orders,
        title=title
    )

    # 5) Hover(툴팁) 포맷
    #    - 발표에서는 ‘필요한 정보만’ 간결히: 선수명, 시즌(정수), 값(포맷 적용).
    #    - legendgroup을 쓰면 범례 그룹 이름(=선수명)이 툴팁에 안정적으로 들어간다.
    if yfmt == "avg":
        # 타율 등 비율형은 소수 셋째 자리(야구 관례).
        hover_tmpl = "<b>%{legendgroup}</b><br>Season: %{x:.0f}<br>Value: %{y:.3f}<extra></extra>"
    else:
        # 카운트형(SO/HR 등)은 소수점 없이 깔끔하게.
        hover_tmpl = "<b>%{legendgroup}</b><br>Season: %{x:.0f}<br>Value: %{y:.0f}<extra></extra>"

    # 6) 트레이스 공통 스타일
    #    - lines+markers: 선+포인트 동시 표기 → 추세와 개별 시즌 값 모두 강조.
    #    - hovertemplate 적용으로 툴팁을 일관되게 커스터마이즈.
    fig.update_traces(mode="lines+markers", hovertemplate=hover_tmpl)

    # 7) 레이아웃(발표 가독성 향상)
    #    - hovermode="x unified": 동일 x에서의 여러 선수 값을 하나의 패널로 묶어 비교 쉬움.
    #    - margin: 제목/축라벨이 잘리지 않게 여백 조정(프로젝터 환경 고려).
    #    - template="plotly_white": 높은 대비 + 미니멀 스타일.
    fig.update_layout(
        hovermode="x unified",
        xaxis_title="Season",
        yaxis_title=ytitle,
        legend_title="Player",
        margin=dict(l=40, r=20, t=60, b=40),
        template="plotly_white"
    )

    # 8) 출력
    #    - 노트북/대화형 환경에선 show()가 즉시 렌더링.
    #    - (참고) Streamlit/Dash 등에서는 return fig로 넘겨 별도 렌더러에 그리는 게 적합.
    fig.show()

    # --- 선택적 개선 포인트(필요하면 해제해서 사용) -------------------------
    # # y축 포맷을 축 라벨에도 반영하고 싶다면:
    # if yfmt == "avg":
    #     fig.update_yaxes(tickformat=".3f")  # 축 눈금도 소수 셋째 자리
    # else:
    #     fig.update_yaxes(tickformat=",.0f") # 천단위 구분 정수
    #
    # # 축 눈금 간격을 연단위로 고정하고 싶다면:
    # # fig.update_xaxes(tickmode="linear", dtick=1)
    #
    # # 노이즈가 많아 추세선이 필요하면(발표용 보조 시각):
    # # fig.add_traces(px.scatter(d, x=COL_YEAR, y=ycol, color="legend_name",
    # #                           trendline="ols").data)


In [5]:
# 전준우 / 정훈 / 한동희 시즌별 요약 표 (2018–2024)
cols, agg = [], {}

if "SO" in target.columns:
    cols.append("SO"); agg["SO"] = "sum"     # 시즌 총 삼진
if "HR" in target.columns:
    cols.append("HR"); agg["HR"] = "sum"     # 시즌 총 홈런
if "AVG" in target.columns:
    cols.append("AVG"); agg["AVG"] = "mean"  # 시즌 평균 타율
if "PA" in target.columns:
    cols.append("PA"); agg["PA"] = "sum"     # 시즌 총 타석(있으면)

summary_tbl = (
    target.groupby(["legend_name", COL_YEAR])[cols]
          .agg(agg)
          .reset_index()
          .sort_values(["legend_name", COL_YEAR])
)

# 보기 좋게 반올림 및 정수/소수 포맷
fmt = {}
if "AVG" in summary_tbl.columns: fmt["AVG"] = "{:.3f}".format
for c in ["SO", "HR", "PA"]:
    if c in summary_tbl.columns: fmt[c] = "{:,.0f}".format

try:
    display(summary_tbl.style.format(fmt))
except Exception:
    # 스타일 지원 안 되는 환경 대비
    display(summary_tbl)


,legend_name,year,SO,HR,AVG,PA
0,Han Dong-Hee,2018,58,4,0.232,226
1,Han Dong-Hee,2019,57,2,0.203,207
2,Han Dong-Hee,2020,97,17,0.278,531
3,Han Dong-Hee,2021,95,17,0.267,496
4,Han Dong-Hee,2022,64,14,0.307,499
5,Han Dong-Hee,2023,58,5,0.223,353
6,Han Dong-Hee,2024,9,0,0.257,36
7,Jeon Jun-woo,2018,82,33,0.342,614
8,Jeon Jun-woo,2019,71,22,0.301,606
9,Jeon Jun-woo,2020,79,26,0.279,628


In [6]:
line_chart_px(COL_SO, "Strikeouts per Season (2018–2024) — Lotte", "SO", yfmt="int")


In [7]:
line_chart_px(COL_HR, "Home Runs per Season (2018–2024) — Lotte", "HR", yfmt="int")


In [8]:
line_chart_px("AVG", "Batting Average per Season (2018–2024) — Lotte", "AVG", yfmt="avg")


In [9]:
print(df.columns.tolist())

['player_name', 'team', 'year', 'home_run_rate', 'label', 'batting_side', 'throwing_hand', 'height', 'weight', 'age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'batting_avg', 'onbase_perc', 'slugging_perc', 'IBB', 'onbase_plus_slugging', 'TB', 'GIDP', 'HBP', 'SH', 'SF']
